In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

# 하이퍼파라미터 설정
batch_size = 128  # 배치 크기를 증가
learning_rate = 0.01  # 초기 학습률 증가
num_epochs = 50  # 에포크 수 증가

# 데이터셋 전처리 및 데이터 증강
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(15),  # 회전 각도 증가
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# CIFAR-10 데이터셋 로드
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# 모델 정의 (ResNet-50 사용, 사전학습 가중치 적용)
model = models.resnet50(pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),  # 중간 레이어 노드 수 증가
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 10)
)

# GPU 사용 가능하면 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수, 옵티마이저, 학습률 스케줄러 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)  # 모멘텀과 가중치 감소 추가
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)

# 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 순전파
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 역전파 및 옵티마이저 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(train_loader)

# 테스트 함수
def test(model, test_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # 예측 및 정확도 계산
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# 모델 학습 및 테스트
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer)
    test_accuracy = test(model, test_loader, criterion)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
    scheduler.step()  # 학습률 업데이트

print("모델 학습 완료!")


100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


c:\ProgramData\Anaconda3\envs\mjjenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\envs\mjjenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\user/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100.0%


Epoch [1/50], Loss: 1.2525, Test Accuracy: 0.7370
Epoch [2/50], Loss: 0.8548, Test Accuracy: 0.7865
Epoch [3/50], Loss: 0.7599, Test Accuracy: 0.7978
Epoch [4/50], Loss: 0.7018, Test Accuracy: 0.8000
Epoch [5/50], Loss: 0.6538, Test Accuracy: 0.8144
Epoch [6/50], Loss: 0.6208, Test Accuracy: 0.8291
Epoch [7/50], Loss: 0.5907, Test Accuracy: 0.8359
Epoch [8/50], Loss: 0.5685, Test Accuracy: 0.8352
Epoch [9/50], Loss: 0.5519, Test Accuracy: 0.8391
Epoch [10/50], Loss: 0.5282, Test Accuracy: 0.8375
Epoch [11/50], Loss: 0.5146, Test Accuracy: 0.8439
Epoch [12/50], Loss: 0.4968, Test Accuracy: 0.8380
Epoch [13/50], Loss: 0.4864, Test Accuracy: 0.8474
Epoch [14/50], Loss: 0.4732, Test Accuracy: 0.8459
Epoch [15/50], Loss: 0.4619, Test Accuracy: 0.8496
Epoch [16/50], Loss: 0.4444, Test Accuracy: 0.8495
Epoch [17/50], Loss: 0.4290, Test Accuracy: 0.8596
Epoch [18/50], Loss: 0.4223, Test Accuracy: 0.8563
Epoch [19/50], Loss: 0.4076, Test Accuracy: 0.8551
Epoch [20/50], Loss: 0.3946, Test Accura